![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


In [48]:
-- Finding top 10 companies with the highest ROI
SELECT 
    c.company, 
	c.country,
	i.industry,
    f.valuation, 
    f.funding, 
    ROUND((f.valuation - f.funding) / NULLIF(f.funding, 0), 2) AS ROI  
FROM funding AS f
JOIN companies AS c USING (company_id)
JOIN industries AS i USING (company_id)
WHERE f.valuation IS NOT NULL AND f.funding > 0 -- Exclude companies with missing data
ORDER BY ROI DESC 
LIMIT 10;


,company,country,industry,valuation,funding,roi
0,Zapier,United States,Internet software & services,4000000000,1000000,3999.0
1,Dunamu,South Korea,Fintech,9000000000,71000000,125.0
2,Workhuman,Ireland,Internet software & services,1000000000,9000000,110.0
3,CFGI,United States,Fintech,2000000000,19000000,104.0
4,Manner,China,Other,1000000000,10000000,99.0
5,DJI Innovations,China,Hardware,8000000000,105000000,75.0
6,GalaxySpace,China,Mobile & telecommunications,1000000000,14000000,70.0
7,Canva,Australia,Internet software & services,40000000000,572000000,68.0
8,Il Makiage,United States,E-commerce & direct-to-consumer,2000000000,29000000,67.0
9,Revolution Precrafted,Philippines,Other,1000000000,15000000,65.0


In [28]:
-- Finding the top industries that produced the most unicorns in 2019, 2020, and 2021

-- Count how many unicorns were created in each industry and finding top 3 industries 
WITH cte AS (
	SELECT i.industry, 
		COUNT (i.company_id) AS num_unicorns
	FROM industries AS i
	JOIN dates AS d 
	USING (company_id)
	WHERE EXTRACT(year FROM d.date_joined) in ('2019', '2020', '2021')
	GROUP BY i.industry
	ORDER BY num_unicorns DESC
	LIMIT 3
	)
,
-- Gets yearly unicorn counts and average valuation per industry 
cte2 AS (
    SELECT i.industry, 
           EXTRACT(YEAR FROM d.date_joined) AS year_became_unicorn, 
           COUNT(i.company_id) AS num_unicorns, 
           ROUND(AVG(f.valuation), 2) AS avg_valuation
    FROM industries AS i
    JOIN dates AS d USING (company_id)
    JOIN funding AS f USING (company_id)
    WHERE EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
    GROUP BY i.industry, year_became_unicorn
)
-- Calculates valuation in billions for top 3 industries based on unicorn count
SELECT cte2.industry, 
       cte2.year_became_unicorn AS year, 
       cte2.num_unicorns, 
       ROUND(AVG(cte2.avg_valuation) / 1000000000, 2) AS average_valuation_billions
FROM cte2 
JOIN (SELECT industry FROM cte ORDER BY num_unicorns DESC LIMIT 3) AS top_industries
ON cte2.industry = top_industries.industry
GROUP BY cte2.industry, cte2.year_became_unicorn, cte2.num_unicorns
ORDER BY cte2.year_became_unicorn DESC, cte2.num_unicorns DESC;
	


,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58


In [33]:
-- Finding the number of unicorns per country
SELECT country, 
       COUNT(company_id) AS num_unicorns 
FROM companies 
GROUP BY country  
ORDER BY num_unicorns DESC
LIMIT 10; 



,country,num_unicorns
0,United States,562
1,China,173
2,India,65
3,United Kingdom,43
4,Germany,26
5,France,24
6,Israel,20
7,Canada,19
8,Brazil,16
9,Singapore,12


In [30]:
-- Finding the number of unicorns per city (Unicorn hubs)
SELECT c.city, c.country,  
       COUNT(c.company_id) AS num_unicorns  
FROM companies AS c  
GROUP BY c.city, c.country  
ORDER BY num_unicorns DESC  
LIMIT 10;

,city,country,num_unicorns
0,San Francisco,United States,152
1,New York,United States,103
2,Beijing,China,63
3,Shanghai,China,44
4,London,United Kingdom,34
5,Bengaluru,India,29
6,Shenzhen,China,19
7,Paris,France,19
8,Palo Alto,United States,18
9,Berlin,Germany,17


In [31]:
-- Calculate the average time (in years) from founding to unicorn status per industry
SELECT i.industry,  
       ROUND(AVG(EXTRACT(YEAR FROM d.date_joined) - d.year_founded)::numeric, 2) AS avg_time_to_unicorn_years  
FROM industries AS i  
JOIN dates AS d USING (company_id)  
WHERE EXTRACT(year FROM d.date_joined) IS NOT NULL  
GROUP BY i.industry  
ORDER BY avg_time_to_unicorn_years ASC;

,industry,avg_time_to_unicorn_years
0,Auto & transportation,5.03
1,Artificial intelligence,5.89
2,Hardware,5.94
3,Mobile & telecommunications,6.32
4,E-commerce & direct-to-consumer,6.43
5,Fintech,6.54
6,Travel,6.57
7,Cybersecurity,6.80
8,"Supply chain, logistics, & delivery",6.93
9,Edtech,7.71


In [32]:
-- Calculate the average growth rate per industry for last 5 years (2017-2021)
WITH unicorn_counts AS (
    -- Count the number of unicorns per industry per year for 2017-2021
    SELECT i.industry,
           EXTRACT(YEAR FROM d.date_joined) AS year,
           COUNT(d.company_id) AS num_unicorns
    FROM industries AS i
    JOIN dates AS d USING (company_id)
    WHERE EXTRACT(YEAR FROM d.date_joined) BETWEEN 2017 AND 2021
    GROUP BY i.industry, year
),
growth_rate AS (
    -- Calculate year-over-year growth rate for each industry from 2017-2021
    SELECT industry,
           year,
           num_unicorns,
           LAG(num_unicorns) OVER (PARTITION BY industry ORDER BY year) AS prev_year_unicorns,
           CASE 
               WHEN LAG(num_unicorns) OVER (PARTITION BY industry ORDER BY year) > 0
               THEN (num_unicorns - LAG(num_unicorns) OVER (PARTITION BY industry ORDER BY year)) * 100.0 / LAG(num_unicorns) OVER (PARTITION BY industry ORDER BY year)
               ELSE NULL
           END AS growth_rate
    FROM unicorn_counts
)
-- Calculate the average growth rate
SELECT industry,
       ROUND (AVG(growth_rate),2) AS avg_growth_rate
FROM growth_rate
WHERE growth_rate IS NOT NULL 
GROUP BY industry
ORDER BY avg_growth_rate DESC;  




,industry,avg_growth_rate
0,Hardware,605.56
1,"Supply chain, logistics, & delivery",511.93
2,Artificial intelligence,313.53
3,Fintech,240.42
4,Cybersecurity,220.24
5,Internet software & services,197.93
6,Health,140.58
7,Consumer & retail,133.33
8,E-commerce & direct-to-consumer,106.77
9,Edtech,100.00
